In [2]:
import numpy as np
import tensorflow as tf
seed = 2022
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

텍스트 전처리


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [7]:
# 특징
# 1. 가나다 순서가 아닌 많이 나온 단어 순서
# 2. 1 에서부터 시작함
# 3. 
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [11]:
# 단어 집합 크기 - Keras Tokenizer의 시작 인덱스가 1이기 때문에 1을 더해주어야 함
vocab_size = len(t.word_index) + 1
vocab_size

12

In [13]:
t.texts_to_sequences(["그의 말이 법이다."])

[[6, 1, 7]]

In [15]:
# corpus를 encoding 한 결과
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [16]:
t.texts_to_sequences([text])

[[2, 3, 1, 4, 5, 6, 1, 7, 8, 1, 9, 10, 1, 11]]

- sequence 생성

In [17]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [19]:
# sequence 중에서 제일 길이가 긴 것
max_len = max(len(seq) for seq in sequences)
max_len

6

- Padding
    - 전체 샘플 길이를 max_len에 맞추도록 0 을 추가
    - 앞쪽을 채우는 pre옵션이 디폴트

In [22]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

- X,y 데이터 설정

In [23]:
X = sequences[:, :-1]
y = sequences[:, -1]
X.shape, y.shape

((11, 5), (11,))

In [24]:
y = to_categorical(y)
y.shape

### 모델
- Embedding
- SimpleRNN
- Dense

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

In [28]:
# Embedding = 4, SimpleRNN = 32
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim = 4, input_length = max_len-1),
    SimpleRNN(32), # activation = tanh 는 default 옵션
    Dense(vocab_size, activation = "softmax")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 4)              48        
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1184      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile("adam", "categorical_crossentropy", ["accuracy"] )

In [39]:
hist = model.fit(X, y, epochs = 200, verbose = 0)

In [40]:
hist.history["accuracy"][-1] # 정확도 100%

1.0

- 모델 검증

In [46]:
from my_util import sentence_generation
sentence_generation(model, t, max_len, "법이다", 5)

'법이다 말이 말이 있는 말이 곱다'

#### 모델 변화
- Embedding vector 갯수 : [2,4,6]
- RNN 유닛 갯수 : [24,32,48]

In [47]:
for n_embed in [2,4,6]:
    for n_unit in [24,32,48]:
        model = Sequential([
                Embedding(input_dim=vocab_size, output_dim = n_embed, input_length = max_len-1),
                SimpleRNN(n_unit), # activation = tanh 는 default 옵션
                Dense(vocab_size, activation = "softmax")
             ])
        model.compile("adam","categorical_crossentropy", ["accuracy"] )
        hist = model.fit(X, y, epochs = 200, verbose = 0)
        print("===============================================")
        print(f"Embedding vector : {n_embed}, RNNunit : {n_unit}")
        print(f"정확도 : {hist.history['accuracy'][-1]:.4f}")
        print(sentence_generation(model, t, max_len, "가는", 7))
        print(sentence_generation(model, t, max_len, "그의", 5))
        print(sentence_generation(model, t, max_len, "경마장에", 6))
        print()

Embedding vector : 2, RNNunit : 24
정확도 : 0.9091
가는 말이 고와야 오는 말이 곱다 오는 말이
그의 말이 법이다 뛰고 있다 곱다
경마장에 말이 말이 뛰고 있다 곱다 뛰고

Embedding vector : 2, RNNunit : 32
정확도 : 0.8182
가는 말이 고와야 오는 말이 곱다 곱다 오는
그의 말이 말이 고와야 오는 말이
경마장에 말이 말이 뛰고 있다 있다 곱다

Embedding vector : 2, RNNunit : 48
정확도 : 0.9091
가는 말이 고와야 오는 말이 곱다 말이 곱다
그의 말이 법이다 말이 말이 오는
경마장에 말이 고와야 오는 말이 곱다 말이

Embedding vector : 4, RNNunit : 24
정확도 : 0.9091
가는 말이 고와야 오는 말이 곱다 있다 말이
그의 말이 고와야 오는 말이 곱다
경마장에 있는 말이 뛰고 있다 오는 곱다

Embedding vector : 4, RNNunit : 32
정확도 : 0.9091
가는 말이 고와야 오는 말이 곱다 있다 말이
그의 말이 법이다 말이 고와야 오는
경마장에 말이 고와야 오는 말이 곱다 있다

Embedding vector : 4, RNNunit : 48
정확도 : 1.0000
가는 말이 고와야 오는 말이 곱다 오는 말이
그의 말이 법이다 말이 말이 오는
경마장에 있는 말이 뛰고 있다 말이 곱다

Embedding vector : 6, RNNunit : 24
정확도 : 1.0000
가는 말이 고와야 오는 말이 곱다 있다 곱다
그의 말이 법이다 고와야 있는 말이
경마장에 있는 말이 뛰고 있다 오는 말이

Embedding vector : 6, RNNunit : 32
정확도 : 1.0000
가는 말이 고와야 오는 말이 곱다 오는 있다
그의 말이 법이다 말이 말이 법이다
경마장에 있는 말이 뛰고 있다 곱다 곱다

Embedding vector : 6, RNNunit : 48
정확도 : 1.0000
가는 말이 고와야 오는 말이 곱다 

1184